In [10]:
from breeze_connect import BreezeConnect
import pandas as pd

key_api = "465431N5PF87604W817T9s827K0P0172"
key_secret = "F`39g946341=P88186BWH3I698Y45689"
key_session ='2357115'

# Initialize SDK
breeze = BreezeConnect(api_key=key_api)
breeze.generate_session(api_secret=key_secret, session_token=key_session)

In [11]:
breeze.get_funds()

{'Success': {'bank_account': '043901004074',
  'total_bank_balance': 2322.82,
  'allocated_equity': 200.32,
  'allocated_fno': 2122.5,
  'block_by_trade_equity': 0.0,
  'block_by_trade_fno': 29.98,
  'block_by_trade_balance': 29.98,
  'unallocated_balance': '156301.53'},
 'Status': 200,
 'Error': None}

In [12]:
import talib
from datetime import datetime
# set the start time as per your wish
start = datetime(2022,12,20,14,13)
end = datetime(2022,12,24,14,59)
now = datetime.now()

In [16]:
data = breeze.get_historical_data(interval="1minute",
                            from_date= "2022-12-10T07:00:00.000Z",
                            to_date= "2022-12-30T07:00:00.000Z",
                            stock_code="ITC",
                            exchange_code="NSE",
                            product_type="cash")
# convert data into dataframe
df = pd.DataFrame(data['Success'])
df = df[['datetime','open','high','low','close']] 

In [13]:
def format_data(df):
    # convert the 'datetime' column to Date format 
    df['datetime']= pd.to_datetime(df['datetime'])
    #convert other columns to number format
    for column in df.columns:
        if column == 'datetime': pass
        else : 
            df[column] = df[column].astype(float)  
            df[column] = df[column].round(1)

    return df

In [14]:
def add_sma(df):
    # calculate moving averages uaing close price - 14 day and 30 day and return the updated dataframe
    df['short_sma'] = df.close.rolling(14).mean().round(1)
    df['long_sma'] = df.close.rolling(30).mean().round(1)
    df = df.dropna()
    return df

In [15]:
def add_rsi(df, period):
  # calculate RSI for 'period' and return the updated dataframe
    delta = df['close'].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    rUp = up.ewm(com=period - 1,adjust=False).mean()
    rDown = down.ewm(com=period - 1,adjust=False).mean().abs()
    df['RSI_' + str(period)] = 100 - 100 / (1 + rUp / rDown)
    df['RSI_' + str(period)].fillna(0,inplace=True)
    df['RSI_' + str(period)] = df['RSI_' + str(period)].round(1)
    df = df.dropna()
    return df

In [ ]:
stock_list = ["GAIL","YESBAN","BHEL","IDECEL","INDR","PUNBAN"]

In [27]:
breeze.get_funds()

{'Success': {'bank_account': '043901004074',
  'total_bank_balance': 2822.82,
  'allocated_equity': 700.32,
  'allocated_fno': 2122.5,
  'block_by_trade_equity': 0.0,
  'block_by_trade_fno': 29.98,
  'block_by_trade_balance': 29.98,
  'unallocated_balance': '155801.53'},
 'Status': 200,
 'Error': None}

In [32]:
stock_list = ["GAIL","YESBAN","BHEL","IDECEL","INDR","PUNBAN"]
from datetime import datetime
from datetime import timedelta
import time
while True:
    for stock in range(len(stock_list)):
        stock_status = {"GAIL":"waiting","YESBAN" : "waiting","BHEL" : "waiting","IDECEL" : "waiting","INDR" :"waiting"}
        data = breeze.get_historical_data(interval="1minute",
                                from_date= "2022-12-10T07:00:00.000Z",
                                to_date= "2023-01-02T07:00:00.000Z",
                                stock_code=stock_list[stock],
                                exchange_code="NSE",
                                product_type="cash")
        # convert data into dataframe
        df = pd.DataFrame(data['Success'])
        df = df[['datetime','open','high','low','close']] 
        format_data(df)
        add_sma(df)
        add_rsi(df,14)
        pr = add_rsi(df,14)
        RSI = pr["RSI_14"].iloc[-1]
        RSI_1 = pr["RSI_14"].iloc[-2]
        if  (RSI_1 >50) and stock_status[stock_list[stock]]== "waiting":
            print(f'current level of{stock_list[stock]} RSI{RSI}')
            print('RSI is greater than 50 :Buy signal')
            buy = breeze.place_order(stock_code=stock_list[stock],
                        exchange_code="NSE",
                        product="cash",
                        action="buy",
                        order_type="market",
                        stoploss="",
                        quantity="1",
                        price=str(current_price),
                        validity="day")
            print(buy)
            stock_status.update({stock_list[stock]:"EXECUTED"})
            print(stock_status[stock_list[stock]])
    
            
        else:
            print(f'current level of {stock_list[stock]} RSI is {RSI}')
            print("RSI is lesser than 50 :DO Nothing")
        price = breeze.get_quotes(stock_code = stock_list[stock],
                                  exchange_code = "NSE",
                                  product_type = "cash")   

        if price['Status'] == 200:
            current_price = price["Success"][0]['ltp']
            print(f'request was a success,current {stock_list[stock]} price is:{current_price}')
        else:
            print('failed request')
        now = datetime.now()
        time.sleep(1)

current level of GAIL RSI is 48.2
RSI is lesser than 50 :DO Nothing
request was a success,current GAIL price is:96.05
current level of YESBAN RSI is 37.7
RSI is lesser than 50 :DO Nothing
request was a success,current YESBAN price is:21.65
current level of BHEL RSI is 52.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:80.7
current level ofIDECEL RSI54.4
RSI is greater than 50 :Buy signal
{'Success': None, 'Status': 500, 'Error': 'Order Value less than minimum order value:Please modify the order'}
EXECUTED
request was a success,current IDECEL price is:7.95
current level of INDR RSI is 50.9
RSI is lesser than 50 :DO Nothing
request was a success,current INDR price is:32.9


KeyError: 'PUNBAN'

In [9]:
# # stock_list = ["GAIL",YESBAN","BHEL","IDECEL","INDR","PUNBAN"]
# from datetime import datetime
# from datetime import timedelta
# import time
# while True: 
#     format_data(df)
#     add_sma(df)
#     add_rsi(df,14)
#     pr = add_rsi(df,14)
#     RSI = pr["RSI_14"].iloc[-1]
#     RSI_1 = pr["RSI_14"].iloc[-2]
#     if (RSI < RSI_1) and (RSI_1 >50) :
#         print(f'current level of RSI{RSI}')
#         print('RSI is greater than 50 :Buy signal')
#         buy = breeze.place_order(stock_code="BHEL",
#                     exchange_code="NSE",
#                     product="cash",
#                     action="buy",
#                     order_type="market",
#                     stoploss="",
#                     quantity="1",
#                     price=str(current_price),
#                     validity="day")
#         print(buy)
#         break
#     else:
#         print(f'current level of{stock_list[stock]} RSI is {RSI}')
#         print("RSI is lesser than 50 :DO Nothing")
#     price = breeze.get_quotes(stock_code = "BHEL",
#                               exchange_code = "NSE",
#                               product_type = "cash")   
   
#     if price['Status'] == 200:
#         current_price = price["Success"][0]['ltp']
#         print(f'request was a success,current BHEL price is:{current_price}')
#     else:
#         print('failed request')
#     now = datetime.now()
#     time.sleep(1)

current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:79.2
current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:79.2
current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:79.2
current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:79.2
current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:79.2
current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:79.2
current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:79.2
current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price is:79.2
current level of RSI is 49.7
RSI is lesser than 50 :DO Nothing
request was a success,current BHEL price 

KeyboardInterrupt: 

In [21]:
breeze.ws_connect()

In [ ]:
# Callback to receive ticks.
def on_ticks(ticks):
    print("Ticks: {}".format(ticks))

# Assign the callbacks.
breeze.on_ticks = on_ticks

# subscribe stocks feeds
breeze.subscribe_feeds(exchange_code="NSE", stock_code="ZEEENT", product_type="cash", get_exchange_quotes=True,get_market_depth = True)

In [ ]:
breeze.unsubscribe_feeds(exchange_code="NSE", stock_code="ZEEENT", product_type="cash", get_exchange_quotes=True, get_market_depth = True)

In [ ]:
breeze.ws_disconnect()

In [ ]:
# Sohum - project
# LIVE RSI deployment for 5 stocks
## replace getquote api and use websockets


In [ ]:

# Sideways - 60%  -  Create good side way Straregies(3)
# Trend following - 3  
# momentum - 3
# Mean reverting - 3
# First -step
# Initial Creating
# combination of technical indictors in market movement 
# Second Step 
# Analyzing the market movement 

